Predizendo Preço de games na Steam

In [527]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  mean_squared_error, mean_absolute_error
import numpy as np


In [528]:
df = pd.read_json('steamdb.json')
df.head(10)

In [529]:
df['published_store'] = pd.to_datetime(df['published_store'])
df.info()

A base de dados têm bastante indices nulos e colunas que não nos dão muita informação como URL do game.
Precisamos retirar itens chave e manter itens que possam ter alguma relevância mas não tenham tantos NaN.

In [530]:

indices_to_drop = [0,1,2,5,6,7,8,9,11,14,15,16,17,18,19,24,26,28,30,32,33,36,37,39,40,41,42,43]

# Obter os nomes das colunas correspondentes aos índices
col_names_to_drop = df.columns[indices_to_drop]

# Remover as colunas
df = df.drop(columns=col_names_to_drop)
df['Idade_do_Produto'] = (datetime.now() - df['published_store']).dt.days
df = df.drop(columns= 'published_store')

In [531]:
df.isna().sum()

In [532]:
df.dropna().shape

In [533]:
df.info()

In [534]:
df.nunique()

Podemos categorizar a dificuldade e tentar expandir os multivalorados 'categories' e 'tags'

In [535]:
df['gfq_difficulty'] = df['gfq_difficulty'].astype('category')

df['name']=df['name'].astype('string')

In [536]:
df['genres'].unique()

In [537]:
df['categories'] = df['categories'].apply(lambda x: x.split(',') if x is not None else [])
df['genres'] = df['genres'].apply(lambda x: x.split(',') if x is not None else [])
df['tags'] = df['tags'].apply(lambda x: x.split(',') if x is not None else [])
df.head()

In [538]:
colunas_para_verificar = ['tags', 'categories', 'genres']

# Contando valores únicos para cada coluna
for coluna in colunas_para_verificar:
    # Achatar a lista de listas
    itens_achatados = [item for sublist in df[coluna].dropna() for item in sublist]

    # Obter os valores únicos
    itens_unicos = set(itens_achatados)

    # Contar e exibir o número de itens únicos
    print(f"Número de itens únicos em '{coluna}': {len(itens_unicos)}")

Temos portanto um grande número de generos e categorias e ainda maior de tags. 
Expandir esses indices pode levar a um número muito extenso de colunas e valores para administrar, isso seria adequado para outros tipos de modelo, diferentes dos que estamos trabalhando na matéria. portanto vamos trabalhar apenas com as outras colunas.

In [539]:
# Instanciar o MultiLabelBinarizer para fazer o one-hot encoding
combined_labels = df.apply(lambda x: set(x['categories'] + x['genres'] + x['tags']), axis=1)

# Instanciar o MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Aplicar o one-hot encoding
combined_encoded = pd.DataFrame(mlb.fit_transform(combined_labels), columns=mlb.classes_, index=df.index)

# Concatenar com o DataFrame original
df_encoded = pd.concat([df, combined_encoded], axis=1)

df_encoded.shape

In [540]:
df_encoded = df_encoded.dropna()
df_encoded = df_encoded.drop(columns='full_price')
df_encoded['Multi'] = df_encoded['Multi-player'] | df_encoded['Multiplayer']
#df_encoded = df_encoded.drop(columns=['Multiplayer', 'Multi-player'])

In [541]:
corr_matrix = df_encoded.corr()

# Filtrar colunas com correlação acima de um limiar (0.1 neste caso)
# Verificar primeiro se as colunas estão presentes no DataFrame
cols_to_keep = corr_matrix.columns[corr_matrix.abs()['current_price'] > 0.15]

# Criar uma nova matriz de correlação apenas com as colunas filtradas
filtered_corr_matrix = df_encoded[cols_to_keep].corr()

# Usar seaborn para criar um heatmap da matriz de correlação filtrada
plt.figure(figsize=(20, 20))
sns.heatmap(filtered_corr_matrix, annot=True, fmt=".2f", cmap='bwr', center= 0.05)

# Mostrar o gráfico
plt.show()

In [542]:
modelos = list(range(0, 39))
r_ajustados_modelo_1 = []
f_statistics_modelo_1 = []
limite = []
for i in range(40):
    corr_matrix = df_encoded.corr(numeric_only=True)
    multi = 0.01 + (0.01 * i)
    limite.append(multi)
    cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > multi]
    cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
    X = df_encoded[cols_to_keep]  # Variáveis independentes
    y = df_encoded['current_price']       # Variável dependente
    X_sm = sm.add_constant(X)
    model_sm = sm.OLS(y, X_sm).fit()


    adj_r_squared = model_sm.rsquared_adj
    f_statistic = model_sm.fvalue

    r_ajustados_modelo_1.append(adj_r_squared)
    f_statistics_modelo_1.append(f_statistic)


# Normalizar os dados
scaler = MinMaxScaler()

# Transformar R² ajustado e F-Statistic
r_ajustados_normalizados = scaler.fit_transform(np.array(r_ajustados_modelo_1).reshape(-1, 1)).flatten()
f_statistics_normalizados = scaler.fit_transform(np.array(f_statistics_modelo_1).reshape(-1, 1)).flatten()

# Calcular taxa de crescimento para os dados normalizados
r_ajustados_crescimento = np.diff(r_ajustados_normalizados) / r_ajustados_normalizados[:-1]
f_statistics_crescimento = np.diff(f_statistics_normalizados) / f_statistics_normalizados[:-1]

# Plotar gráficos
plt.figure(figsize=(12, 6))

# Curva de crescimento de R² Ajustado
plt.plot(modelos, r_ajustados_crescimento, label='Crescimento R² Ajustado Normalizado', color='blue', marker='o')

# Curva de crescimento de F-Statistic
plt.plot(modelos, f_statistics_crescimento, label='Crescimento F-Statistic Normalizado', color='red', marker='x')

plt.xlabel('Número do Modelo')
plt.ylabel('Taxa de Crescimento Normalizada')
plt.title('Taxa de Crescimento Normalizada do R² Ajustado e F-Statistic')
plt.legend()
plt.grid(True)
plt.show()

In [543]:
teste_p = set()
limite_p_valor = 0.05
for i in [0.05 , 0.1, 0.15, 0.2, 0.25, 0.3, 0.35]:
    corr_matrix = df_encoded.corr(numeric_only=True)
    cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > i]
    cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
    X = df_encoded[cols_to_keep]  # Variáveis independentes
    y = df_encoded['current_price']       # Variável dependente
    X_sm = sm.add_constant(X)
    model_sm = sm.OLS(y, X_sm).fit()

    
    adj_r_squared = model_sm.rsquared_adj
    f_statistic = model_sm.fvalue
    

# Iterar sobre os pares de nomes de variáveis e seus valores-p
    for variavel, p_valor in model_sm.pvalues.items():
        if p_valor > limite_p_valor:
            teste_p.add(variavel)

print("Variáveis com valor-p alto:", teste_p)


0.08 0.16 0.24 0.35

corr |0.06 |0.17 |0.25 |0.35
R aj |54   |49   |47   |44
F-S  |28   |122  |221  |490

In [544]:
df_encoded = df_encoded.drop(columns=list(teste_p))
#df_encoded = df_encoded.drop(columns=['hltb_complete','Multiplayer', 'Puzzle', 'PvP'])

In [545]:
r_ajustados_modelo_2 = []
f_statistics_modelo_2 = []
for i in range(40):
    corr_matrix = df_encoded.corr(numeric_only=True)
    multi = 0.01 + (0.01 * i)
    cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > multi]
    cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
    X = df_encoded[cols_to_keep]  # Variáveis independentes
    y = df_encoded['current_price']       # Variável dependente
    X_sm = sm.add_constant(X)
    model_sm = sm.OLS(y, X_sm).fit()


    adj_r_squared = model_sm.rsquared_adj
    f_statistic = model_sm.fvalue

    r_ajustados_modelo_2.append(adj_r_squared)
    f_statistics_modelo_2.append(f_statistic)
    
scaler = MinMaxScaler()

# Transformar R² ajustado e F-Statistic
r_ajustados_normalizados = scaler.fit_transform(np.array(r_ajustados_modelo_2).reshape(-1, 1)).flatten()
f_statistics_normalizados = scaler.fit_transform(np.array(f_statistics_modelo_2).reshape(-1, 1)).flatten()

# Calcular taxa de crescimento para os dados normalizados
r_ajustados_crescimento = np.diff(r_ajustados_normalizados) / r_ajustados_normalizados[:-1]
f_statistics_crescimento = np.diff(f_statistics_normalizados) / f_statistics_normalizados[:-1]

# Plotar gráficos
plt.figure(figsize=(12, 6))

# Curva de crescimento de R² Ajustado
plt.plot(modelos, r_ajustados_crescimento, label='Crescimento R² Ajustado Normalizado', color='blue', marker='o')

# Curva de crescimento de F-Statistic
plt.plot(modelos, f_statistics_crescimento, label='Crescimento F-Statistic Normalizado', color='red', marker='x')

plt.xlabel('Número do Modelo')
plt.ylabel('Taxa de Crescimento Normalizada')
plt.title('Taxa de Crescimento Normalizada do R² Ajustado e F-Statistic')
plt.legend()
plt.grid(True)
plt.show()

In [546]:
modelos = list(range(len(r_ajustados_modelo_1)))
modelos = [x/100 for x in modelos]
# Plotar gráficos
plt.figure(figsize=(12, 6))

# Comparação de R² Ajustado
plt.subplot(1, 2, 1)
plt.plot(modelos, r_ajustados_modelo_1, label='R² Ajustado Modelo 1', color='blue', marker='o')
plt.plot(modelos, r_ajustados_modelo_2, label='R² Ajustado Modelo 2', color='green', marker='x')
plt.xlabel('Correlação mínima')
plt.ylabel('R² Ajustado')
plt.title('Comparação de R² Ajustado entre Modelos')
plt.legend()

# Comparação de F-Statistic
plt.subplot(1, 2, 2)
plt.plot(modelos, f_statistics_modelo_1, label='F-Statistic Modelo 1', color='red', marker='o')
plt.plot(modelos, f_statistics_modelo_2, label='F-Statistic Modelo 2', color='orange', marker='x')
plt.xlabel('Correlação mínima')
plt.ylabel('F-Statistic')
plt.title('Comparação de F-Statistic entre Modelos')
plt.legend()

plt.tight_layout()
plt.show()

Agora que podemos observar claramente a diferença entre os mais diversos limites de correlação para adicão de variáveis dependentes no modelo
e após termos retirado todas as colunas que poderiam ter casos de insignificância, podemos prosseguir testando polinômios de graus maiores e seus resíduos.

### Testar limites em graus diferentes:

In [547]:

corr_matrix = df_encoded.corr()
r_ajustados_poli_corr_2=[]
r_ajustados_poli_corr_27=[]
f_statistics_poli_corr_2=[]
f_statistics_poli_corr_27=[]
for i in [0.2,0.27]:
    cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > i]
    cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
    X = df_encoded[cols_to_keep]  # Variáveis independentes
    y = df_encoded['current_price']
    
    for degree in range(1, 5):  # Graus 1 a 5
        poly = PolynomialFeatures(degree)
        X_poly = poly.fit_transform(X)

        X_sm_poly = sm.add_constant(X_poly)
        model_sm_poly = sm.OLS(y, X_sm_poly).fit()

        # Extrair R² ajustado e F-statistic
        adj_r_squared = model_sm_poly.rsquared_adj
        f_statistic = model_sm_poly.fvalue

        if i<0.27:
            r_ajustados_poli_corr_2.append(adj_r_squared)
            f_statistics_poli_corr_2.append(f_statistic)
        else:
            r_ajustados_poli_corr_27.append(adj_r_squared)
            f_statistics_poli_corr_27.append(f_statistic)

modelos = list(range(1,len(r_ajustados_poli_corr_2)+1))          
plt.figure(figsize=(12, 6))

# Comparação de R² Ajustado
plt.subplot(1, 2, 1)
plt.plot(modelos, r_ajustados_poli_corr_2, label='R² Ajustado corr 0.2', color='blue', marker='o')
plt.plot(modelos, r_ajustados_poli_corr_27, label='R² Ajustado corr 0.27', color='green', marker='x')
plt.xlabel('Grau Polinomial')
plt.ylabel('R² Ajustado')
plt.title('Comparação de R² Ajustado entre Modelos')
plt.legend()

# Comparação de F-Statistic
plt.subplot(1, 2, 2)
plt.plot(modelos, f_statistics_poli_corr_2, label='F-Statistic corr 0.2', color='blue', marker='o')
plt.plot(modelos, f_statistics_poli_corr_27, label='F-Statistic corr 0.27', color='green', marker='x')
plt.xlabel('Grau polinomial')
plt.ylabel('F-Statistic')
plt.title('Comparação de F-Statistic entre Modelos')
plt.legend()

plt.tight_layout()
plt.show()
            


A julgar pelos resultados acredito que o limite de 0.27 seja melhor ajustado em segundo grau, sacrificando um pouco de generalização por significância
mas não perdendo tanta generalização. qualquer coisa além de um polinomio de segundo grau parece perder muito. vejamos um exemplo extremo de cada. max de R ajustado, Max de F-stat e meio termo.

In [548]:
# Plotar gráficos lado a lado
plt.figure(figsize=(15, 5))

cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > 0.2]
cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['current_price']

poly = PolynomialFeatures(4)
X_poly1 = poly.fit_transform(X)

X_sm_poly1 = sm.add_constant(X_poly1)
model_sm_poly1 = sm.OLS(y, X_sm_poly1).fit()

adj_r_squared = model_sm_poly1.rsquared_adj
f_statistic = model_sm_poly1.fvalue

# Extrair e imprimir os valores-p
print(f"Resultados do Modelo - Grau {4}:\n")
print(f"R² Ajustado: {adj_r_squared}")
print(f"F-Statistic: {f_statistic}\n")

y_pred1 = model_sm_poly1.predict(X_sm_poly1)


# Gráfico para o Modelo 1
plt.subplot(1, 3, 1)  # 1 linha, 3 colunas, 1º gráfico
plt.scatter(y, y_pred1)
plt.xlabel('Valores Observados')
plt.ylabel('Valores Previstos')
plt.title('Modelo Grau 4')

cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > 0.27]
cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['current_price']

poly = PolynomialFeatures(1)
X_poly2 = poly.fit_transform(X)

X_sm_poly2 = sm.add_constant(X_poly2)
model_sm_poly2 = sm.OLS(y, X_sm_poly2).fit()

adj_r_squared = model_sm_poly2.rsquared_adj
f_statistic = model_sm_poly2.fvalue

# Extrair e imprimir os valores-p
print(f"Resultados do Modelo - Grau {1}:\n")
print(f"R² Ajustado: {adj_r_squared}")
print(f"F-Statistic: {f_statistic}\n")

y_pred2 = model_sm_poly2.predict(X_sm_poly2)

# Gráfico para o Modelo 2
plt.subplot(1, 3, 2)  # 1 linha, 3 colunas, 2º gráfico
plt.scatter(y, y_pred2)
plt.xlabel('Valores Observados')
plt.ylabel('Valores Previstos')
plt.title('Modelo Grau 1')


poly = PolynomialFeatures(2)
X_poly3 = poly.fit_transform(X)

X_sm_poly3 = sm.add_constant(X_poly3)
model_sm_poly3 = sm.OLS(y, X_sm_poly3).fit()

adj_r_squared = model_sm_poly3.rsquared_adj
f_statistic = model_sm_poly3.fvalue

# Extrair e imprimir os valores-p
print(f"Resultados do Modelo - Grau {2}:\n")
print(f"R² Ajustado: {adj_r_squared}")
print(f"F-Statistic: {f_statistic}\n")

y_pred1 = model_sm_poly1.predict(X_sm_poly1)
y_pred3 = model_sm_poly3.predict(X_sm_poly3)

# Gráfico para o Modelo 3
plt.subplot(1, 3, 3)  # 1 linha, 3 colunas, 3º gráfico
plt.scatter(y, y_pred3)
plt.xlabel('Valores Observados')
plt.ylabel('Valores Previstos')
plt.title('Modelo Grau 2')

plt.tight_layout()
plt.show()

## PUTZ estou Lascado, claramente tanto os dados, quanto os residuos seguem um padrão de heterocedasticidade
##### quase todos os valores parecem agir de maneira categórica, podemos obeservar nas previsões e valores reais, que o preço
##### parece se manter em regiões como 3000, 4000, 5000. O modelo não se adequa bem para esse tipo de dado, talvez seja melhor 
##### fazer um modelo logistico e tentar prever de maneira categórica.

## **TESTE DE PREVISÕES E RESÍDUOS** 

In [549]:
residuos1 = y - y_pred1
residuos2 = y - y_pred2
residuos3 = y - y_pred3
# Modelo 1
shapiro_test1 = stats.shapiro(residuos1)
print("Shapiro-Wilk Modelo 1:", shapiro_test1)

# Modelo 2
shapiro_test2 = stats.shapiro(residuos2)
print("Shapiro-Wilk Modelo 2:", shapiro_test2)

# Modelo 3
shapiro_test3 = stats.shapiro(residuos3)
print("Shapiro-Wilk Modelo 3:", shapiro_test3)

In [550]:
# Modelo 1
bp_test1 = sm.stats.diagnostic.het_breuschpagan(residuos1, model_sm_poly1.model.exog)
print("Breusch-Pagan Modelo 1:", bp_test1)

# Modelo 2
bp_test2 = sm.stats.diagnostic.het_breuschpagan(residuos2, model_sm_poly2.model.exog)
print("Breusch-Pagan Modelo 2:", bp_test2)

# Modelo 3
bp_test3 = sm.stats.diagnostic.het_breuschpagan(residuos3, model_sm_poly3.model.exog)
print("Breusch-Pagan Modelo 3:", bp_test3)

In [551]:
# Substitua 'caminho_para_salvar/arquivo.csv' pelo caminho e nome do arquivo desejado
df_encoded.to_csv('arquivo.csv', index=False)


Primeiramente vamos ver a distribuição de current_price, devia ter visto desde o início.

In [521]:
plt.figure(figsize=(20, 10))
df_encoded['current_price'].value_counts().sort_index().plot(kind='bar')
plt.xlabel('Preço Atual')
plt.ylabel('Contagem')
plt.title('Distribuição de Current Price')
plt.show()

Podemos observar claramente que a distribuição dos valores se concentram em alguns valores, portanto estipulemos um número mínimo de ocorrências para
fazer parte de uma coluna categorizada desses valores.

In [522]:
bins = [0, 499, 999, 1499, 1999, 6000]
labels = ['0-499', '500-999', '1000-1499', '1500-1999', '2000+']

# Categorizando os preços
df_encoded['current_price'] = pd.cut(df_encoded['current_price'], bins=bins, labels=labels, right=False)

# Verificar a distribuição das novas categorias
print(df_encoded['current_price'].value_counts())
plt.figure(figsize=(20, 10))
df_encoded['current_price'].value_counts().sort_index().plot(kind='bar')
plt.xlabel('Preço Atual')
plt.ylabel('Contagem')
plt.title('Distribuição de Current Price')
plt.show()

podemos testar o modelo logistico com esse número de categorias

In [523]:
df_encoded['current_price'] = df_encoded['current_price'].astype('category')
df_encoded['current_price'].info()
df_encoded['current_price'].nunique()

In [526]:
cols_to_keep = corr_matrix.index[corr_matrix.abs()['current_price'] > 0.1]
cols_to_keep = cols_to_keep.drop('current_price')  # Remover a coluna 'score' da lista
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['current_price']

# Adicionando uma constante ao modelo
X = sm.add_constant(X)

modelo_logistico_multinomial = sm.MNLogit(y, X).fit()

# Exibir o sumário do modelo
print(modelo_logistico_multinomial.summary())

In [492]:
df_encoded[cols_to_keep].describe()

In [493]:
for variavel in cols_to_keep:
    # Calcular Q1 (25º percentil) e Q3 (75º percentil)
    Q1 = df_encoded[variavel].quantile(0.25)
    Q3 = df_encoded[variavel].quantile(0.75)

    # Calcular o IQR (intervalo interquartil)
    IQR = Q3 - Q1

    # Definir limites para identificar outliers
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Filtrar outliers
    df_encoded = df_encoded[(df_encoded[variavel] >= limite_inferior) & (df_encoded[variavel] <= limite_superior)]

# Verificar o resultado após a remoção de outliers
print(df_encoded[cols_to_keep].describe())

In [494]:
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['current_price']

# Adicionando uma constante ao modelo
X = sm.add_constant(X)

modelo_logistico_multinomial = sm.MNLogit(y, X).fit()

# Exibir o sumário do modelo
print(modelo_logistico_multinomial.summary())

In [496]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Ajustar o modelo de regressão logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Avaliar o modelo
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))